Need to check different approaches- oversampling, xgboost, NNets etc to see if model can be further improved

### Module imports and data

#### Imports

In [1]:
import pandas as pd
import numpy as np
import xgboost
from time import time
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix
from skopt.space import Real, Categorical, Integer
from skopt import gp_minimize, gbrt_minimize, forest_minimize, BayesSearchCV
import datetime
pd.options.display.max_rows = 1000
login_reference_date = datetime.datetime(2020, 3, 22)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
mds = pd.read_pickle('/home/ubuntu/shubham_gandhi/datasets/master_data_w_embdng_2020_10_01.pkl')

#### Select model columns

In [3]:
model_cols = ['num_bank_accounts', 'countof_distinct_book_id', 'countof_distinct_ifsc_codes', 'fraction_of_deleted_books', 'number_deleted_books', 'countof_bank_name', 'num_uniq_customers', 'total_links_sent', 'total_links_sent_to_others', 'fraction_of_links_sent_to_self', 'status_two_to_one_ratio', 'ttl_pymnt_lnks_amount_sent', 'ttl_sccssfl_pymnt_lnks_amount_sent', 'fraction_successsful_amt_sent', 'num_pymnt_lnks_rcvd_frm_othrs', 'ttl_pymnt_lnks_amount_rcvd', 'ttl_sccssfl_pymnt_lnks_amount_rcvd', 'fraction_successsful_amt_rcvd', 'distinct_trxn_days', 'num_unique_merchants', 'num_unique_customers', 'median_num_links_sent', 'num_uniq_kb_merchants_on_ntwrk', 'num_links_to_kb_merchants', 'median_links_to_kb_merchant', 'ttl_pymnt_lnks_amount_sent_kb_ntwrk', 'ttl_sccssfl_pymnt_lnks_amount_sent_kb_ntwrk', 'fraction_successsful_amt_sent_kb_ntwrk',  'num_customer_is_kb_mrchnt', 'ttotal_links_sent_to_himself', 'median_time_taken_to_send_another_link', 'rolling_2_transactions_median', 'rolling_2_transactions_mean', 'rolling_3_transactions_median', 'rolling_3_transactions_mean', 'rolling_6_transactions_median', 'rolling_6_transactions_mean', 'rolling_10_transactions_median', 'rolling_10_transactions_mean', 'rolling_15_transactions_median', 'rolling_15_transactions_mean', 'rolling_21_transactions_median', 'rolling_21_transactions_mean', 'rolling_28_transactions_median', 'rolling_28_transactions_mean', 'rolling_54_transactions_median', 'rolling_54_transactions_mean', 'cnt_sccss_trxn', 'has_pan', 'has_address', 'has_gst', 'has_dl', 'has_mobile', 'has_elec', 'countof_names', 'countof_pans', 'countof_dobs', 'countof_doc_types', 'num_doc_submissions', 'time_to_first_payment_link', 'time_to_activation', 'time_to_power', 'time_to_first_cust', 'time_to_first_trxn', 'app_age_of_user', 'status_two_to_one_ratio_within_48hrs', 'status_two_to_one_ratio_within_4hrs', 'status_two_to_one_ratio_within_6hrs', 'status_two_to_one_ratio_within_10hrs', 'status_two_to_one_ratio_within_24hrs', 'total_links_sent_within_48hrs', 'total_links_sent_within_4hrs', 'total_links_sent_within_6hrs', 'total_links_sent_within_10hrs', 'total_links_sent_within_24hrs', 'total_links_sent_to_others_within_48hrs', 'total_links_sent_to_others_within_4hrs', 'total_links_sent_to_others_within_6hrs', 'total_links_sent_to_others_within_10hrs', 'total_links_sent_to_others_within_24hrs', 'num_unique_customers_within_48hrs', 'num_unique_customers_within_4hrs', 'num_unique_customers_within_6hrs', 'num_unique_customers_within_10hrs', 'num_unique_customers_within_24hrs', 'fraction_of_links_sent_to_self_within_48hrs', 'fraction_of_links_sent_to_self_within_4hrs', 'fraction_of_links_sent_to_self_within_6hrs', 'fraction_of_links_sent_to_self_within_10hrs', 'fraction_of_links_sent_to_self_within_24hrs', 'ttl_pymnt_lnks_amount_sent_within_48hrs', 'ttl_sccssfl_pymnt_lnks_amount_sent_within_48hrs', 'fraction_successsful_amt_sent_within_48hrs', 'ttl_pymnt_lnks_amount_sent_within_4hrs', 'ttl_sccssfl_pymnt_lnks_amount_sent_within_4hrs', 'fraction_successsful_amt_sent_within_4hrs', 'ttl_pymnt_lnks_amount_sent_within_6hrs', 'ttl_sccssfl_pymnt_lnks_amount_sent_within_6hrs', 'fraction_successsful_amt_sent_within_6hrs', 'ttl_pymnt_lnks_amount_sent_within_10hrs', 'ttl_sccssfl_pymnt_lnks_amount_sent_within_10hrs', 'fraction_successsful_amt_sent_within_10hrs', 'ttl_pymnt_lnks_amount_sent_within_24hrs', 'ttl_sccssfl_pymnt_lnks_amount_sent_within_24hrs', 'fraction_successsful_amt_sent_within_24hrs', 'median_time_taken_to_send_another_link_within_4hrs', 'median_time_taken_to_send_another_link_within_6hrs', 'median_time_taken_to_send_another_link_within_10hrs', 'median_time_taken_to_send_another_link_within_24hrs', 'median_time_taken_to_send_another_link_within_48hrs', 'status_two_to_one_ratio_within_72hrs', 'total_links_sent_within_72hrs', 'total_links_sent_to_others_within_72hrs', 'num_unique_customers_within_72hrs', 'fraction_of_links_sent_to_self_within_72hrs', 'ttl_pymnt_lnks_amount_sent_within_72hrs', 'ttl_sccssfl_pymnt_lnks_amount_sent_within_72hrs', 'fraction_successsful_amt_sent_within_72hrs', 'median_time_taken_to_send_another_link_within_72hrs', 'median_amt_of_transactions_within_4hrs', 'mean_amt_of_transactions_within_4hrs', 'median_amt_of_transactions_within_6hrs', 'mean_amt_of_transactions_within_6hrs', 'median_amt_of_transactions_within_10hrs', 'mean_amt_of_transactions_within_10hrs', 'median_amt_of_transactions_within_24hrs', 'mean_amt_of_transactions_within_24hrs', 'median_amt_of_transactions_within_48hrs', 'mean_amt_of_transactions_within_48hrs', 'median_amt_of_transactions_within_72hrs', 'mean_amt_of_transactions_within_72hrs', 'status_two_to_one_ratio_within_168hrs', 'total_links_sent_within_168hrs', 'total_links_sent_to_others_within_168hrs', 'num_unique_customers_within_168hrs', 'fraction_of_links_sent_to_self_within_168hrs', 'ttl_pymnt_lnks_amount_sent_within_168hrs', 'ttl_sccssfl_pymnt_lnks_amount_sent_within_168hrs', 'fraction_successsful_amt_sent_within_168hrs', 'median_time_taken_to_send_another_link_within_168hrs', 'median_amt_of_transactions_within_168hrs', 'mean_amt_of_transactions_within_168hrs', 'num_books', 'cust_names_gibberish_percentage', 'cust_names_num_gibberish_names', 'pymt_names_num_gibberish_names', 'num_pymt_links', 'pymt_names_gibberish_percentage', 'median_pymt_names_score', 'median_cust_names_score', 'rgn_embdg_0', 'rgn_embdg_1', 'rgn_embdg_2', 'rgn_embdg_3', 'rgn_embdg_4', 'rgn_embdg_5', 'rgn_embdg_6', 'rgn_embdg_7','rgn_embdg_8', 'rgn_embdg_9', 'rgn_embdg_10', 'rgn_embdg_11', 'rgn_embdg_12', 'rgn_embdg_13', 'rgn_embdg_14'] + ['cty_embdg_0', 'cty_embdg_1', 'cty_embdg_2', 'cty_embdg_3','cty_embdg_4', 'cty_embdg_5', 'cty_embdg_6', 'cty_embdg_7','cty_embdg_8']

In [4]:
mds.limit_bucket_id = mds.limit_bucket_id.astype(str)
categorical_features = ['region', 'city', "limit_bucket_id"]
mds[["has_pan", "has_address", "has_gst", "has_dl", "has_mobile", "has_elec"]] = mds[["has_pan", "has_address", "has_gst", "has_dl", "has_mobile", "has_elec"]].astype(bool)

mds_ = mds.sample(frac=1, random_state=42).copy()
mds_[categorical_features] = mds_[categorical_features].fillna("")
# mds_.loc[mds_.in_pymts_data, :]

mds_.in_pymts_data.sum()
mds_.in_kyc.sum()

mds_[["in_kyc", "in_pymts_data"]] = mds_[["in_kyc", "in_pymts_data"]].astype(bool)
mds_[mds_.in_pymts_data].fraud_flag.sum()
sum(mds.first_payments_link_date > login_reference_date)

mds_ = mds_[(mds_.in_pymts_data) & (mds_.in_kyc) & ( (((mds_.fraud_flag==1) | (mds_.fraud_flag==2)) & (mds_.first_payments_link_date>login_reference_date))| (mds_.fraud_flag==0))]
mds_.groupby('fraud_flag')['first_payments_link_date'].min()
mds_.shape



(88656, 221)

#### Deselect columns with too many nulls 

In [5]:
acceptable_null_cols = mds_.isnull().sum()[mds_.isnull().sum() < 5000].index.tolist()
acceptable_null_cols = [i for i in acceptable_null_cols if i in model_cols]

In [6]:
len(acceptable_null_cols)

165

In [7]:
print(mds_[acceptable_null_cols].dropna().shape)
print(mds_.shape)

(85142, 165)
(88656, 221)


#### Split data in train and test

In [8]:
target_col = "fraud_flag"
model_cols = acceptable_null_cols
keep_index = mds_[model_cols].dropna().index
model_cols_data = mds_.loc[keep_index, model_cols].reset_index().drop("index", axis=1)
model_cols_labels = mds_.loc[keep_index, target_col].reset_index().drop("index", axis=1)

model_cols_labels.fraud_flag = model_cols_labels.fraud_flag.astype(int)
train_index_frac = 0.7
model_cols_train_index_limit = int(train_index_frac*model_cols_data.shape[0])
model_cols_train_data = model_cols_data[:model_cols_train_index_limit]
model_cols_train_labels = model_cols_labels[:model_cols_train_index_limit]
model_cols_test_data = model_cols_data[model_cols_train_index_limit:]
model_cols_test_labels = model_cols_labels[model_cols_train_index_limit:]
model_cols_train_data.shape

(59599, 165)

#### Define f1_scorer for BayesSearchCV

In [9]:
def f1_bscv(estimator, X, y):
    y_pred = estimator.predict(X)
    score =  f1_score(y, y_pred, average="macro")
    with open("f1_scores.txt", "a") as g:
        g.write("score: {}\n".format(score))
    return score

### XGBoost

#### Train XGBoost

In [22]:

xgb_space = {
    "XGB__n_estimators": Integer(10, 20),
    "XGB__max_depth": Integer(6, 8),
    "XGB__gamma": Real(0.01, 1, prior='log-uniform'),
    "XGB__learning_rate": Real(0.0001, 1, prior='log-uniform'),
    "XGB__max_delta_step": Real(0.1, 10, prior='log-uniform'),
    "XGB__reg_alpha": Real(0.1, 100, prior='log-uniform'),
    "XGB__reg_lambda": Real(0.1, 100, prior='log-uniform'),
    "XGB__subsample": Real(0.3, 0.7, prior='log-uniform'),
#     "scale_pos_weight": Real(272, 278, prior='log-uniform')
}


n_iter=10

pipe = Pipeline([
    ("Standard_Scaler", StandardScaler(with_mean=True, with_std=True)), 
    ("XGB", xgboost.XGBClassifier(random_state=42, objective="multi:softmax",num_classes=3))
])

xgb_opt = BayesSearchCV(
    pipe,
    xgb_space,
    n_iter=n_iter,
    random_state=42, n_points=1, cv=2, n_jobs=-1,
    scoring=f1_bscv, refit=True
)

t0 = time()
_ = xgb_opt.fit(model_cols_train_data, model_cols_train_labels)
print("XGB F1 after model tuning: ", xgb_opt.score(model_cols_test_data, model_cols_test_labels))
print(time()-t0)



/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[15:25:04] WARNING: /workspace/src/learner.cc:480: 
Parameters: { num_classes } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


XGB F1 after model tuning:  0.4880419064405404
132.66030836105347


##### Evaluate Precision-Recall of best iteration

In [23]:

from sklearn.metrics import confusion_matrix, recall_score, precision_score
print("Recall for XGB: ",recall_score(model_cols_test_labels, xgb_opt.predict(model_cols_test_data), average="macro"))
print("Precision for XGB: ",precision_score(model_cols_test_labels, xgb_opt.predict(model_cols_test_data), average="macro"))
print("F1 for XGB: ",f1_score(model_cols_test_labels, xgb_opt.predict(model_cols_test_data), average="macro"))
confusion_matrix(model_cols_test_labels, xgb_opt.predict(model_cols_test_data))


Recall for XGB:  0.4284466700306049
Precision for XGB:  0.7728362840523513
F1 for XGB:  0.4880419064405404


array([[25125,    31,     1],
       [  316,    50,     0],
       [   12,     5,     3]])

#### Train XGB with all fraud labels as one

Currently we are performing multiclass classification by using `fraud_flag = 2` for chargebacks and `fraud_flag = 1` for risk ops behavior. I want to try with binary classification, like earlier

##### Create labels for binary classification

In [17]:
model_cols_train_labels_bnry = model_cols_train_labels.copy()
model_cols_test_labels_bnry = model_cols_test_labels.copy()

In [18]:
model_cols_train_labels_bnry[model_cols_train_labels_bnry.fraud_flag==2] = 1
model_cols_test_labels_bnry[model_cols_test_labels_bnry.fraud_flag==2] = 1

##### Binary classification xgboost

Note: _Doesnt help much_

In [21]:

xgb_space = {
    "n_estimators": Integer(10, 20),
    "max_depth": Integer(6, 8),
    "gamma": Real(0.01, 1, prior='log-uniform'),
    "learning_rate": Real(0.0001, 1, prior='log-uniform'),
    "max_delta_step": Real(0.1, 10, prior='log-uniform'),
    "reg_alpha": Real(0.1, 100, prior='log-uniform'),
    "reg_lambda": Real(0.1, 100, prior='log-uniform'),
    "subsample": Real(0.3, 0.7, prior='log-uniform'),
    "scale_pos_weight": Real(1, 278, prior='log-uniform')
}


n_iter=100

pipe = Pipeline([
    ("Standard_Scaler", StandardScaler(with_mean=True, with_std=True)), 
    ("XGB", xgboost.XGBClassifier(random_state=42, objective="multi:softmax",num_classes=3))
])

xgb_opt = BayesSearchCV(
    pipe,
    xgb_space,
    n_iter=n_iter,
    random_state=42, n_points=1, cv=2, n_jobs=-1,
    scoring=f1_bscv, refit=True
)


t0 = time()
_ = xgb_opt.fit(model_cols_train_data, model_cols_train_labels_bnry)
print("XGB F1 after model tuning: ", xgb_opt.score(model_cols_test_data, model_cols_test_labels_bnry))
print(time()-t0)



/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[04:33:37] WARNING: /workspace/src/learner.cc:480: 
Parameters: { num_classes } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


XGB F1 after model tuning:  0.39691289966923926
1329.4521310329437


### CatBoost

#### Modify CatBoost

I need to modify CatBoost because I need to use multi class weights and reformat before BayesSearch calls the `set_params` method

In [10]:
import inspect, sys, json
from six import iteritems, string_types, integer_types
class ModCatBoostClassifier(CatBoostClassifier, object):
    def set_params(self, **params):
#         a
        params = locals()['params']
        params = {k:params[k] for k in params if k not in ['self']}
        print(params)
        if "class_weights_1" in params:
            params["class_weights"] = [1, params["class_weights_1"], params["class_weights_2"]]
            del params["class_weights_1"]
            del params["class_weights_2"]
        with open("file2.txt", "a") as f:
            for item in list(params.keys()):
                f.write("{}: {}\n".format(item, params[item]))
        return super().set_params(**params)

    

#### Train CatBoost

In [ ]:

from skopt.space import Real, Integer, Categorical
import catboost as cb
from catboost import Pool

cb_space = {
    "CB__iterations": Integer(120, 500),
    "CB__depth": Integer(8, 16),
    "CB__learning_rate": Real(0.3, 1, prior='log-uniform'),
    "CB__l2_leaf_reg": Real(10, 30, prior='log-uniform'),
    'CB__random_strength': Real(1, 10, 'log-uniform'), 
    "CB__class_weights_1": Real(0.5, 500, name="class_weights_1"),
    "CB__class_weights_2": Real(0.5, 500, name="class_weights_2"),
}

n_iter=10

pipe = Pipeline([
    ("Standard_Scaler", StandardScaler(with_mean=True, with_std=True)), 
    ("CB", ModCatBoostClassifier(random_seed=42, logging_level='Verbose', boosting_type="Plain", 
                                  border_count=120, custom_metric="F1:use_weights=True", 
                                  bootstrap_type="Bayesian", metric_period=50, od_wait=50, od_type="Iter"))
])


cb_opt = BayesSearchCV(pipe,
    cb_space,
    n_iter=n_iter,
    optimizer_kwargs={'base_estimator': 'GP'},
    random_state=42, n_points=5, cv=3, n_jobs=-1,
    scoring=f1_bscv, refit=True
)

# train_dataset = Pool(model_cols_train_data, model_cols_train_labels)
# train_dataset.quantize()

from time import time
t0 = time()
_ = cb_opt.fit(model_cols_train_data, model_cols_train_labels)
print("CB F1 after model tuning: ", cb_opt.score(model_cols_test_data, model_cols_test_labels))
print(time()-t0)


In [124]:
cm = pd.DataFrame(confusion_matrix(model_cols_test_labels, cb_opt.predict(model_cols_test_data)))
cm.index = ["Actual_Flag_"+str(i) for i in range(3)]
cm.columns = ["Predicted_Flag_"+str(i) for i in range(3)]
cm.loc[:, "total"] = model_cols_test_labels.fraud_flag.value_counts().values
cm

,Predicted_Flag_0,Predicted_Flag_1,Predicted_Flag_2,total
Actual_Flag_0,24713,441,3,25157
Actual_Flag_1,180,184,2,366
Actual_Flag_2,8,9,3,20


### Neural Nets

#### Data prep

Tensorflow doesnt accept booleans so convert booleans to `np.float32`

In [10]:
bool_types = model_cols_train_data.columns[model_cols_train_data.dtypes == bool].tolist()
model_cols_train_data[bool_types] = model_cols_train_data[bool_types].astype(np.float32)
model_cols_test_data[bool_types] = model_cols_test_data[bool_types].astype(np.float32)

model_cols_train_labels.fraud_flag = model_cols_train_labels.fraud_flag.astype(np.float32)
model_cols_test_labels.fraud_flag = model_cols_test_labels.fraud_flag.astype(np.float32)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


##### Dummy encode the labels

In [11]:
from keras.utils import np_utils
dummy_y = np_utils.to_categorical(model_cols_labels['fraud_flag'].values)
dummy_y_tr = np_utils.to_categorical(model_cols_train_labels['fraud_flag'].values)
dummy_y_te = np_utils.to_categorical(model_cols_test_labels['fraud_flag'].values)
 

#### Keras Model Evaluator

In [12]:
def f1_macro(y_true, y_pred):
    print("F1 _MACRO")
    print(type(y_true))
    print(y_true.shape)
    y_true = y_true.numpy()
    print("F1 _MACRO END!!!!")
    return f1_score(y_true, y_pred, average="macro")



#### Baseline NN

In [13]:
import pandas
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [14]:
# define baseline model
def get_model():
    # create model
    model = Sequential()
    model.add(Dense(170, input_dim=model_cols_train_data.shape[1], activation='relu'))
    model.add(Dropout(0.2))
#     model.add(Dense(500, input_dim=model_cols_train_data.shape[1], activation='swish'))
#     model.add(Dense(200, input_dim=model_cols_train_data.shape[1], activation=tf.nn.leaky_relu))
#     model.add(Dense(40, activation='relu'))
#     model.add(Dense(10, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=[tfa.metrics.F1Score(num_classes=3, average="macro")]
                 )
    return model

#### Define the stratified k-folds 

In [15]:
kfolds = list(StratifiedKFold(n_splits=10, shuffle=True).split(model_cols_data, model_cols_labels))
model = get_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 170)               28220     
_________________________________________________________________
dropout (Dropout)            (None, 170)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 513       
Total params: 28,733
Trainable params: 28,733
Non-trainable params: 0
_________________________________________________________________


#### Train Model

In [79]:
from time import time
epochs = 10
batch_size = 1024
X_train = model_cols_data
y_train = dummy_y
tr_scores = []
te_scores = []
t0 = time()
for j, (train_idx, val_idx) in enumerate(kfolds):
    print('\nFold ',j)
    X_train_cv = X_train.loc[train_idx, :]
    y_train_cv = y_train[train_idx]
    X_valid_cv = X_train.loc[val_idx, :]
    y_valid_cv= y_train[val_idx]
    ss = StandardScaler()
    X_train_cv = ss.fit_transform(X_train_cv)
    X_valid_cv = ss.transform(X_valid_cv)
    model = get_model()
    model.fit(X_train_cv, y_train_cv, epochs = epochs, batch_size = batch_size, validation_data = (X_valid_cv, y_valid_cv), verbose=0)
    tr_scores.append(model.evaluate(X_train_cv, y_train_cv)[-1])
    te_scores.append(model.evaluate(X_valid_cv, y_valid_cv)[-1])
#     print("train set: ", tr_scores[-1])
#     print("test  set: ", te_scores[-1])

print("\\\\\\\\n")
t1 = time()
print("batch_size: ", batch_size)
print("epochs: ", epochs)
print("time: ", round(t1-t0), " seconds")
print("mean train score: ", sum(tr_scores)/len(tr_scores))
print("mean test  score: ", sum(te_scores)/len(te_scores))


Fold  0
267/267 [==============================] - 0s 445us/step - loss: 0.0513 - f1_score: 0.5136

Fold  1
267/267 [==============================] - 0s 514us/step - loss: 0.0488 - f1_score: 0.5234

Fold  2
267/267 [==============================] - 0s 476us/step - loss: 0.0521 - f1_score: 0.5920

Fold  3
267/267 [==============================] - 0s 503us/step - loss: 0.0503 - f1_score: 0.4365

Fold  4
267/267 [==============================] - 0s 487us/step - loss: 0.0489 - f1_score: 0.4475

Fold  5
267/267 [==============================] - 0s 510us/step - loss: 0.0534 - f1_score: 0.4762

Fold  6
267/267 [==============================] - 0s 475us/step - loss: 0.0491 - f1_score: 0.4518

Fold  7
267/267 [==============================] - 0s 496us/step - loss: 0.0585 - f1_score: 0.4873

Fold  8
267/267 [==============================] - 0s 456us/step - loss: 0.0542 - f1_score: 0.4685

Fold  9
267/267 [==============================] - 0s 503us/step - loss: 0.0516 - f1_score: 0.6067


- adding standard scaler step has helped improve results
- adding drop out has helped reduce overfitting
- batch size, epoch

2HL
HL: (200,40); dropout: 0.2, relu
batch_size:  1024
epochs:  10
time:  19  seconds
mean train score:  0.6199480772018433
mean test  score:  0.43590298295021057

1 HL
HL: 170, dropout: 0.2, relu
batch_size:  1024
epochs:  10
time:  17  seconds
mean train score:  0.6465416669845581
mean test  score:  0.5217714965343475

HL: 200, dropout: 0.2, relu
batch_size:  1024
epochs:  10
time:  18  seconds
mean train score:  0.6403773069381714
mean test  score:  0.49829577207565307

HL: 200, dropout: 0.2, relu
batch size: 2 
epoch: 10
mean train score:  0.4366429090499878
mean test  score:  0.41368075013160704

#### Perform tuning for epochs and batch size

In [23]:
from time import time

batch_size = [512, 1024, 2048, 4096, 8192, 16384, 32768]
epochs = [10, 20, 50]

X_train = model_cols_data
y_train = dummy_y
cv_scores = []
t0 = time()
for bs in batch_size:
    for ep in epochs:
        tr_scores = []
        te_scores = []
        print("\n")
        print("|"*80)
        print("\nbatch_size: {} epochs: {}".format(bs, ep))
        for j, (train_idx, val_idx) in enumerate(kfolds):
            print('\nFold ',j)
            X_train_cv = X_train.loc[train_idx, :]
            y_train_cv = y_train[train_idx]
            X_valid_cv = X_train.loc[val_idx, :]
            y_valid_cv= y_train[val_idx]
            ss = StandardScaler()
            X_train_cv = ss.fit_transform(X_train_cv)
            X_valid_cv = ss.transform(X_valid_cv)
            model = get_model()
            model.fit(X_train_cv, y_train_cv, epochs = ep, batch_size = bs, validation_data = (X_valid_cv, y_valid_cv), verbose=0)
            tr_scores.append(model.evaluate(X_train_cv, y_train_cv)[-1])
            te_scores.append(model.evaluate(X_valid_cv, y_valid_cv)[-1])
        cv_scores.append({
            "batch_size":bs,
            "epochs":ep,
            "train_score": sum(tr_scores)/len(tr_scores),
            "test_score": sum(te_scores)/len(te_scores),
        })





||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

batch_size: 512 epochs: 10

Fold  0


KeyboardInterrupt: 

In [ ]:
cv_scores

In [24]:
max([i["test_score"] for i in cv_scores])

ValueError: max() arg is an empty sequence

In [25]:
{'batch_size': 512,
  'epochs': 50,
  'train_score': 0.7758227586746216,
  'test_score': 0.5248038798570633},

({'batch_size': 512,
  'epochs': 50,
  'train_score': 0.7758227586746216,
  'test_score': 0.5248038798570633},)

#### Perform a Grid Search for Dropout and number of neurons in hidden layer

In [26]:
def get_model(hs, dp):
    # create model
    model = Sequential()
    model.add(Dense(hs, input_dim=model_cols_train_data.shape[1], activation='relu'))
    model.add(Dropout(dp))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=[tfa.metrics.F1Score(num_classes=3, average="macro")]
                 )
    return model


In [29]:
from time import time

hl_size = list(range(50, 501, 50))
drop_outs = [i/100 for i in range(5, 26, 5)]
X_train = model_cols_data
y_train = dummy_y
cv_scores = []
t0 = time()
for hs in hl_size:
    for dp in drop_outs:
        tr_scores = []
        te_scores = []
        print("\n")
        print("|"*80)
        print("\nhidden_layer: {} dropout: {}".format(hs, dp))
        for j, (train_idx, val_idx) in enumerate(kfolds):
            print('\nFold ',j)
            X_train_cv = X_train.loc[train_idx, :]
            y_train_cv = y_train[train_idx]
            X_valid_cv = X_train.loc[val_idx, :]
            y_valid_cv= y_train[val_idx]
            ss = StandardScaler()
            X_train_cv = ss.fit_transform(X_train_cv)
            X_valid_cv = ss.transform(X_valid_cv)
            model = get_model(hs, dp)
            model.fit(X_train_cv, y_train_cv, epochs = 10, batch_size = 1024, validation_data = (X_valid_cv, y_valid_cv), verbose=0)
            tr_scores.append(model.evaluate(X_train_cv, y_train_cv)[-1])
            te_scores.append(model.evaluate(X_valid_cv, y_valid_cv)[-1])
        cv_scores.append({
            "layer_size":hs,
            "dropout":dp,
            "train_score": sum(tr_scores)/len(tr_scores),
            "test_score": sum(te_scores)/len(te_scores),
        })





||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

hidden_layer: 50 dropout: 0.05

Fold  0
267/267 [==============================] - 0s 478us/step - loss: 0.0536 - f1_score: 0.4250

Fold  1
267/267 [==============================] - 0s 485us/step - loss: 0.0554 - f1_score: 0.4157

Fold  2
267/267 [==============================] - 0s 452us/step - loss: 0.0519 - f1_score: 0.5676

Fold  3


KeyboardInterrupt: 

In [ ]:
cv_scores

In [15]:
hl_size = list(range(50, 501, 50))
drop_outs = [i/100 for i in range(5, 21, 5)]
len(drop_outs)*len(hl_size)

40

### Oversampling

#### Modify oversampling classes

In [11]:
from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN, BorderlineSMOTE
import inspect, sys, json
from six import iteritems, string_types, integer_types

class ModRandomOverSampler(RandomOverSampler):
    def set_params(self, **params):
        params = locals()['params']
        params = {k:params[k] for k in params if k not in ['self']}
        print(params)
        if "zeros" in params:
            sampling_strategy = {0:params["zeros"], 1:params["ones"], 2:params["twos"]}
            del params["zeros"], params["ones"], params["twos"]
            params['sampling_strategy'] = sampling_strategy
        with open("mos_ros_file.txt", "a") as f:
            for item in list(params.keys()):
                f.write("\n\n ModROS params")
                f.write("{}: {}\n".format(item, params[item]))
        return super().set_params(**params)
        


#### Oversampling Strategy

We have tried a few different approaches to get a good model fit. The last remaining measure I want to implement is Oversampling. By oversampling the minority classes, we may get an improvement on the model fit. 

I'll be using methods from the `imbalanced-learn` module to do this. Since we have three classes in our target column, we need to decide what the resampled distribution of the classes will look like. Lets begin by looking at our current distribution

In [14]:
model_cols_labels.fraud_flag.value_counts()*100/model_cols_labels.shape[0]

0    98.491931
1     1.411759
2     0.096310
Name: fraud_flag, dtype: float64

With ones below 1.5 percent and twos below 0.1 percent, I will be resampling these classes to be 10 and 5 percent. For each fold (fraction=0.2), assuming stratified sampling, we'll get 240 cases of ones and 16 cases of twos and 67k cases of zeros. 

$$67086/(67086+960*x+65*y) = 0.85$$

$$960*x/(67086+960*x+65*y) = 0.1$$

$$65*y/(67086+960*x+65*y) = 0.05$$

$$x = 8.221$$
$$y = 60.773$$

#### Data Preparation

In [12]:
target_col = "fraud_flag"
model_cols = acceptable_null_cols
keep_index = mds_[model_cols].dropna().index
model_cols_data = mds_.loc[keep_index, model_cols].reset_index().drop("index", axis=1)
model_cols_labels = mds_.loc[keep_index, target_col].reset_index().drop("index", axis=1)
model_cols_labels.fraud_flag = model_cols_labels.fraud_flag.astype(int)
train_index_frac = 0.9
model_cols_train_index_limit = int(train_index_frac*model_cols_data.shape[0])
model_cols_train_data = model_cols_data[:model_cols_train_index_limit]
model_cols_train_labels = model_cols_labels[:model_cols_train_index_limit]
model_cols_test_data = model_cols_data[model_cols_train_index_limit:]
model_cols_test_labels = model_cols_labels[model_cols_train_index_limit:]
model_cols_train_data.shape

(76627, 165)

In [13]:
resampled_zeros = 67090
resampled_ones = 7892
resampled_twos = 3947
# resampled_zeros/(resampled_zeros+resampled_ones+resampled_twos)

In [14]:
kfolds = list(StratifiedKFold(n_splits=5, shuffle=True).split(model_cols_data, model_cols_labels))

NameError: name 'StratifiedKFold' is not defined

#### CatBoost on oversampled data

##### Random oversampling

###### Without tuning

In [82]:
from time import time
from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN, BorderlineSMOTE
from sklearn.utils import class_weight
X_train = model_cols_data
y_train = model_cols_labels.fraud_flag
tr_scores = []
te_scores = []
params = {'class_weights': [1,102.06152211981771, 292.60781818845425], 'depth': 12, 
          'iterations': 126, 'l2_leaf_reg': 10.673951271303793, 'learning_rate': 0.6294923985256603, 
          'random_strength': 6.353112028498213
}
# class_weights = [1, 102.06152211981771, 292.60781818845425]
t0 = time()
for j, (train_idx, val_idx) in enumerate(kfolds):
    print('\nFold ',j)
    X_train_cv = X_train.loc[train_idx, :]
    y_train_cv = y_train[train_idx]
    X_valid_cv = X_train.loc[val_idx, :]
    y_valid_cv= y_train[val_idx]
    # standard scale
    ss = StandardScaler()
    X_train_cv = ss.fit_transform(X_train_cv)
    X_valid_cv = ss.transform(X_valid_cv)
    # oversample
    ros = RandomOverSampler(sampling_strategy={0:resampled_zeros,1:resampled_ones,2:resampled_twos}, random_state=42)
    X_train_cv, y_train_cv = ros.fit_resample(X_train_cv, y_train_cv)
    # train model
    class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train_cv), y_train_cv)
    cboost = CatBoostClassifier(random_seed=42, logging_level='Silent', boosting_type="Plain", 
                                  border_count=120, custom_metric="F1:use_weights=True", class_weights=class_weights,
                                   bootstrap_type="Bayesian", metric_period=50, od_wait=50, od_type="Iter")
    params['class_weights'] = class_weights
    cboost.set_params(**params)
    cboost.fit(X_train_cv, y_train_cv)
    tr_scores.append(f1_score(y_train_cv, cboost.predict(X_train_cv), average="macro"))
    te_scores.append(f1_score(y_valid_cv, cboost.predict(X_valid_cv), average="macro"))
    print(tr_scores[-1])
    print(te_scores[-1])

t1 = time()
print("time: ", round(t1-t0), " seconds")
print("mean train score: ", sum(tr_scores)/len(tr_scores))
print("mean test  score: ", sum(te_scores)/len(te_scores))


Fold  0


/home/ubuntu/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:326: UserWarning: After over-sampling, the number of samples (67090) in class 0 will be larger than the number of samples in the majority class (class #0 -> 67086)
  n_samples_majority,
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1 2], y=0        0
1        0
2        0
3        0
4        0
        ..
78924    2
78925    2
78926    2
78927    2
78928    2
Name: fraud_flag, Length: 78929, dtype: int64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


0.9819712711722679
0.5770395447134515

Fold  1


/home/ubuntu/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:326: UserWarning: After over-sampling, the number of samples (67090) in class 0 will be larger than the number of samples in the majority class (class #0 -> 67086)
  n_samples_majority,
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1 2], y=0        0
1        0
2        0
3        0
4        0
        ..
78924    2
78925    2
78926    2
78927    2
78928    2
Name: fraud_flag, Length: 78929, dtype: int64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


0.982889011585038
0.5872160268442966

Fold  2


...

0.9806562894619639
0.5489846634891647

Fold  3


/home/ubuntu/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:326: UserWarning: After over-sampling, the number of samples (67090) in class 0 will be larger than the number of samples in the majority class (class #0 -> 67087)
  n_samples_majority,
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1 2], y=0        0
1        0
2        0
3        0
4        0
        ..
78924    2
78925    2
78926    2
78927    2
78928    2
Name: fraud_flag, Length: 78929, dtype: int64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


0.9845260634058546
0.5492601443813713

Fold  4


/home/ubuntu/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:326: UserWarning: After over-sampling, the number of samples (67090) in class 0 will be larger than the number of samples in the majority class (class #0 -> 67087)
  n_samples_majority,
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1 2], y=0        0
1        0
2        0
3        0
4        0
        ..
78924    2
78925    2
78926    2
78927    2
78928    2
Name: fraud_flag, Length: 78929, dtype: int64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


0.9842875912183214
0.546943302239726
time:  303  seconds
mean train score:  0.9828660453686892
mean test  score:  0.561888736333602


In [85]:
class_weights


array([0.39215482, 3.33371347, 6.66573769])

###### with mod ROS

In [20]:

from skopt.space import Real, Integer, Categorical
import catboost as cb
from catboost import Pool
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN, BorderlineSMOTE

cb_space = {
    # resampled numbers
    "MROS__zeros": Integer(76000, 80000),
    "MROS__ones": Integer(1800, 10000),
    "MROS__twos": Integer(125, 10000),
    # catboost params
    "CB__iterations": Integer(120, 1500),
    "CB__depth": Integer(4, 10),
    "CB__learning_rate": Real(0.3, 1, prior='log-uniform'),
    "CB__l2_leaf_reg": Real(10, 30, prior='log-uniform'),
    'CB__random_strength': Real(1, 10, 'log-uniform'), 
    "CB__class_weights_1": Real(0.5, 500, name="class_weights_1"),
    "CB__class_weights_2": Real(0.5, 500, name="class_weights_2"),
}


pipe = Pipeline([
    ("Standard_Scaler", StandardScaler(with_mean=True, with_std=True)), 
    ("MROS", ModRandomOverSampler(random_state=42)),
    ("CB", ModCatBoostClassifier(random_seed=42, logging_level='Verbose', boosting_type="Plain", 
                                  border_count=120, custom_metric="F1:use_weights=True", 
                                  bootstrap_type="Bayesian", metric_period=50, od_wait=50, od_type="Iter"))
])


n_iter=40
cb_opt = BayesSearchCV(pipe,
    cb_space,
    n_iter=n_iter,
    optimizer_kwargs={'base_estimator': 'GP', 
                      "acq_func_kwargs": {"xi":0.0001, "kappa": 0.0001}},
    random_state=42, n_points=1, cv=5, n_jobs=-1,
    scoring=f1_bscv, refit=True,
)

i = 1
# callback handler
def on_step(optim_result):
    global i
    score = cb_opt.best_score_
    last_tr_score = cb_opt.cv_results_['mean_train_score']
    last_te_score = cb_opt.cv_results_['mean_test_score']    
    print("="*80)
    print("\nIteration num: ", i)
    print("best score: %s" % score)
    print("mean_train_score: %s" % last_tr_score)
    print("mean_test_score: %s" % last_te_score)
    print("\n")
    i+=1

from time import time
t0 = time()
_ = cb_opt.fit(model_cols_train_data, model_cols_train_labels, callback=on_step)
print(time()-t0)



Iteration num:  1
best score: 0.4586384518414529
mean_train_score: []
mean_test_score: [0.4586384518414529]



Iteration num:  2
best score: 0.4586384518414529
mean_train_score: []
mean_test_score: [0.4586384518414529, 0.44125594525975126]



Iteration num:  3
best score: 0.4586384518414529
mean_train_score: []
mean_test_score: [0.4586384518414529, 0.44125594525975126, 0.4319760089484466]



Iteration num:  4
best score: 0.4586384518414529
mean_train_score: []
mean_test_score: [0.4586384518414529, 0.44125594525975126, 0.4319760089484466, 0.44500545175969963]



Iteration num:  5
best score: 0.46210766903482037
mean_train_score: []
mean_test_score: [0.4586384518414529, 0.44125594525975126, 0.4319760089484466, 0.44500545175969963, 0.46210766903482037]



Iteration num:  6
best score: 0.46210766903482037
mean_train_score: []
mean_test_score: [0.4586384518414529, 0.44125594525975126, 0.4319760089484466, 0.44500545175969963, 0.46210766903482037, 0.3819348021124365]



Iteration num:  7
bes

/home/ubuntu/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:326: UserWarning: After over-sampling, the number of samples (76000) in class 0 will be larger than the number of samples in the majority class (class #0 -> 75474)
  n_samples_majority,


0:	learn: 0.8876706	total: 29.6ms	remaining: 44.3s
50:	learn: 0.2051129	total: 1.24s	remaining: 35.2s
100:	learn: 0.0807443	total: 2.49s	remaining: 34.5s
150:	learn: 0.0579838	total: 3.72s	remaining: 33.3s
200:	learn: 0.0473119	total: 4.95s	remaining: 32s
250:	learn: 0.0406808	total: 6.18s	remaining: 30.7s
300:	learn: 0.0361511	total: 7.41s	remaining: 29.5s
350:	learn: 0.0323795	total: 8.66s	remaining: 28.3s
400:	learn: 0.0297274	total: 9.88s	remaining: 27.1s
450:	learn: 0.0272938	total: 11.1s	remaining: 25.9s
500:	learn: 0.0255213	total: 12.4s	remaining: 24.6s
550:	learn: 0.0236171	total: 13.6s	remaining: 23.4s
600:	learn: 0.0220145	total: 14.9s	remaining: 22.2s
650:	learn: 0.0205449	total: 16.2s	remaining: 21.1s
700:	learn: 0.0193036	total: 17.4s	remaining: 19.8s
750:	learn: 0.0183247	total: 18.6s	remaining: 18.6s
800:	learn: 0.0175103	total: 19.9s	remaining: 17.4s
850:	learn: 0.0166417	total: 21.2s	remaining: 16.1s
900:	learn: 0.0158613	total: 22.4s	remaining: 14.9s
950:	learn: 0.01

In [21]:
print("CB F1 after model tuning cmplt: ", cb_opt.score(model_cols_data, model_cols_labels))
print("CB F1 after model tuning train: ", cb_opt.score(model_cols_train_data, model_cols_train_labels))
print("CB F1 after model tuning  test: ", cb_opt.score(model_cols_test_data, model_cols_test_labels))

# CB F1 after model tuning:  0.7851129725982773
# {'class_weights_1': 205.34692744723026, 'class_weights_2': 363.9990087170739, 'depth': 15, 'iterations': 240, 'l2_leaf_reg': 20.880544999616877, 'learning_rate': 0.49391826396295263, 'random_strength': 2.243527177854895}

CB F1 after model tuning cmplt:  0.785643459937655
CB F1 after model tuning train:  0.8154375662580952
CB F1 after model tuning  test:  0.4858220490192151


In [22]:
cb_opt.cv_results_

defaultdict(list,
            {'split0_test_score': [0.380853993955925,
              0.3933664447146025,
              0.37548946943565625,
              0.386174407698159,
              0.40058900677589576,
              0.346165117812891,
              0.4427680581455727,
              0.4630386115919994,
              0.3865054492390965,
              0.4685140012642539,
              0.44609307748466326,
              0.4716693597881916,
              0.47308203991130826,
              0.4924443420529038,
              0.4374143073577923,
              0.47652853111079824,
              0.43306083238503,
              0.46414953825085914,
              0.47586755045649504,
              0.4598226940563431,
              0.49932603158412436,
              0.5140008054334776,
              0.5060945802700647,
              0.5140084649972585,
              0.49198222398405994,
              0.43155093476017736,
              0.4590837107903553,
              0.47887547135445035,
   

In [23]:
cb_opt

BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('Standard_Scaler', StandardScaler()),
                                        ('MROS',
                                         ModRandomOverSampler(random_state=42)),
                                        ('CB',
                                         <__main__.ModCatBoostClassifier object at 0x7efd0e855bd0>)]),
              n_iter=40, n_jobs=-1,
              optimizer_kwargs={'acq_func_kwargs': {'kappa': 0.0001,
                                                    'xi': 0.0001},
                                'base_estimator': 'GP'},
              random_state=42, scoring=<function f1_bscv at 0x7efca...
                             'CB__learning_rate': Real(low=0.3, high=1, prior='log-uniform', transform='identity'),
                             'CB__random_strength': Real(low=1, high=10, prior='log-uniform', transform='identity'),
                             'MROS__ones': Integer(low=1800, high=10000, prior='uniform'

#### NN

In [81]:
from sklearn.utils import class_weight
from time import time
epochs = 10
batch_size = 1024
X_train = model_cols_data
y_train = dummy_y
tr_scores = []
te_scores = []
t0 = time()

for j, (train_idx, val_idx) in enumerate(kfolds):
    print('\nFold ',j)
    X_train_cv = X_train.loc[train_idx, :]
    y_train_cv = y_train[train_idx]
    X_valid_cv = X_train.loc[val_idx, :]
    y_valid_cv= y_train[val_idx]
    labels_train_cv = model_cols_labels.fraud_flag[train_idx]
    labels_valid_cv = model_cols_labels.fraud_flag[val_idx]
    # scale the data
    ss = StandardScaler()
    X_train_cv = ss.fit_transform(X_train_cv)
    X_valid_cv = ss.transform(X_valid_cv)
    # oversample
    ros = RandomOverSampler(sampling_strategy={0:resampled_zeros,1:resampled_ones,2:resampled_twos}, random_state=42)
    X_train_cv, y_train_cv = ros.fit_resample(X_train_cv, y_train_cv)
    # keras model
    model = get_model()
    model.fit(X_train_cv, y_train_cv, epochs = epochs, batch_size = batch_size, 
              validation_data = (X_valid_cv, y_valid_cv), verbose=0)
    tr_scores.append(model.evaluate(X_train_cv, y_train_cv)[-1])
    te_scores.append(model.evaluate(X_valid_cv, y_valid_cv)[-1])
#     print("train set: ", tr_scores[-1])
#     print("test  set: ", te_scores[-1])

t1 = time()
print("batch_size: ", batch_size)
print("epochs: ", epochs)
print("time: ", round(t1-t0), " seconds")
print("mean train score: ", sum(tr_scores)/len(tr_scores))
print("mean test  score: ", sum(te_scores)/len(te_scores))


Fold  0


/home/ubuntu/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:326: UserWarning: After over-sampling, the number of samples (67090) in class 0 will be larger than the number of samples in the majority class (class #0 -> 67086)
  n_samples_majority,
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1 2] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1 2], y=0        0
1        0
2        0
3        0
4        0
        ..
85136    0
85137    0
85138    0
85139    0
85140    0
Name: fraud_flag, Length: 68113, dtype: int64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()